## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-30-18-49-46 +0000,nyt,Experts Question Denmark’s Vaccine Program as ...,https://www.nytimes.com/2025/12/29/world/europ...
1,2025-12-30-18-49-19 +0000,cbc,Green Party settles with commission over exclu...,https://www.cbc.ca/news/politics/green-party-s...
2,2025-12-30-18-49-18 +0000,nypost,3 hikers confirmed dead after going missing on...,https://nypost.com/2025/12/30/us-news/3-hikers...
3,2025-12-30-18-47-00 +0000,wsj,Comex Gold Settles 1.00% Higher at $4370.10,https://www.wsj.com/finance/commodities-future...
4,2025-12-30-18-44-14 +0000,nyt,Saudis Say Airstrike in Yemen Targeted Arms Fr...,https://www.nytimes.com/2025/12/30/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
228,trump,34
170,new,17
87,ukraine,13
328,california,11
329,china,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2025-12-29-23-26-00 +0000,wsj,Trump Threatens New Military Action With Israe...,https://www.wsj.com/world/middle-east/trump-to...,80
124,2025-12-30-10-03-33 +0000,cbc,Trump says U.S. struck dock in Venezuela where...,https://www.cbc.ca/news/world/venezuela-drone-...,75
227,2025-12-29-21-55-33 +0000,wapo,Tensions flare between Russia and Ukraine afte...,https://www.washingtonpost.com/world/2025/12/2...,71
237,2025-12-29-21-12-47 +0000,nyt,"For Zelensky, Just Keeping Trump Talking About...",https://www.nytimes.com/2025/12/29/world/europ...,70
168,2025-12-30-03-22-12 +0000,bbc,'No worries': Trump downplays Chinese military...,https://www.bbc.com/news/articles/cdxwxkeg9w6o...,69


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,80,2025-12-29-23-26-00 +0000,wsj,Trump Threatens New Military Action With Israe...,https://www.wsj.com/world/middle-east/trump-to...
21,53,2025-12-30-17-54-18 +0000,nypost,Misspelled Minnesota day care closed last week...,https://nypost.com/2025/12/30/us-news/misspell...
41,43,2025-12-30-17-11-13 +0000,nypost,Video shows wild brawl moments before 16-year-...,https://nypost.com/2025/12/30/us-news/video-sh...
173,42,2025-12-30-02-37-22 +0000,cbc,China launches live-firing drills around Taiwa...,https://www.cbc.ca/news/world/china-taiwan-war...
197,41,2025-12-30-00-12-41 +0000,bbc,Russia's losses in Ukraine rise faster than ev...,https://www.bbc.com/news/articles/c62n922dnw7o...
124,33,2025-12-30-10-03-33 +0000,cbc,Trump says U.S. struck dock in Venezuela where...,https://www.cbc.ca/news/world/venezuela-drone-...
152,30,2025-12-30-05-14-28 +0000,bbc,Bangladesh's first female prime minister Khale...,https://www.bbc.com/news/articles/cm2v256vdkyo...
254,27,2025-12-29-20-01-01 +0000,nypost,Desperate job seekers use dating apps to get i...,https://nypost.com/2025/12/29/business/despera...
5,26,2025-12-30-18-40-04 +0000,nypost,"Thanks, we hate it here: NY, NJ top list of st...",https://nypost.com/2025/12/30/lifestyle/people...
199,26,2025-12-29-23-59-26 +0000,latimes,"SoCal tennis star's death 'haunts' mom, who vo...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
